# Data cleaning

## SQLite
### CSV datasets
CSV datasets have been imported as SQLite table using PyCharm. Importing a CSV dataset rows that have malformed data are automatically dropped, this is time efficient because no script is needed.

### JSON datasets
Unfortunately PyCharm does not provide a direct import for JSON files. Only one dataset (crypto_telegram_groups) is in json, so a dedicated scprit ([json2db.py](./json2db.py)) has been wrote to import it in the database.

## Data tweaks
### Twitter
Twitter datasets contain some data about november 2020, but that are removed to avoid data interpolation in the charts.

### Datetime re-formatting
Datetimes in the datasets have been re-formatted to a common format 'Year-Month-Day Hours:Minutes:Seconds'. This process has been done by [datetimeformatter.py](./datetimeformatter.py). The process is simply:
- database lines are read one by one
- if the raw datetime matches listed patterns, it is converted using them. In case of no matches, dateutil package is used to recognize pattern and get formatted datetime
- database line is updated using formatted datetime
- 'date' column is populated removing time from formatted datetime

# News crawling

Some news in the final dataset have been crawled from google news using 'Google News' from Hurin Hu. For each day:
- 'Google News' scraps first news page
- 'newspaper' library recover all informations about each article
- the results are saved in a csv

The complete code can be found in [googlecrawler.py](./googlecrawler.py).

Note: the complete csv is next imported in a SQLite to be merged with others news datasets.

# Text classification

Texts have been classified Flair pre-trained english classifier.
Due to time constraints the fast-classifier has been choosed.
Standard classifier is 10 times slower than the fast one (more or less) and with 26 million of tweeter this difference means 240 hours vs 24 hours.

Fast-classifier is a RNN that uses Word2Vec for words embedding.

Each text is classified by [flairclassifier.py](./flairclassifier.py) returning a label and a confidence.

### Label
Label che assume one of three values:
- POSITIVE
- NEUTRAL
- NEGATIVE

NEUTRAL has a double meaning:
- neutral text
- failed classification

### Confidence
Confidence is a value in [0, 1] and rappresents the polarity of sentiment and the reliability of the classification.

### Signed score
Sentiment and confidence could be condensed in one number between -1 and 1, left half for a negative classification and right half for a positive one.

# Datasets merging

To merge datasets about the same media (like, all datasets about twitter)two scripts are been used:

## Generic merger
[csvmerger.py](./csvmerger.py) is the first script that has been wrote to merge database tables. It works but it needs to be re-wrote, in orign it was wrote to merge csvs and than it war adapted to merge databases, so there is code junk and stufs like that. It was used to merge twitter's and telegram's datasets.

## News merger
[newsmerger.py](./newsmerger.py) has been used to merge news datasets; it's a re-wrote version of csvmerger.py but specific for the news; the priciples are the same, but has fewer code junk and it's easier to read.

## How datasets have been merged
Given a list of database's table and a target table the script:
- read line by line each table
- if a line is already in the target table, the scipt "join" the lines trying to complete missing values
- else the script inserts the table's line in the targe one

# Tweets words count
[tweetwordscounter.py](./tweetwordscounter.py) is the scruipt that counts how many words there is a tweet.
This value is inserted in the database for later use, for now, it's just used to calculate average tweets length of each of the users.

# Common colors helper
[palette.py](./palette.py) acts like a wrapper of a dictionary.
Importing palette dictionary inside of palette.py the same color palette can be used in all notebooks, without using HEX values; colors can be simply used accessing the dictiory with keys like 'positive', 'negative', 'price', 'twitter', 'smooth_news', 'strong_telegram' and so on.

An important upgrade could be the creation of anothe palette colorblind-safe.

# Data grouping

## Daily info
Grouping by day is made using SQL:
- all data in a table is grouped by date and label with a query
- for each group, grouping columns are also selected as part of new lines
- each line is corredated with the group lines count and average confidence

`
SELECT date, label, COUNT(ROWID) as count, AVG(conf) as conf FROM tweet GROUP BY date, label;
`

## Users retweets
Also the csv about retweets for each user was made using SQL:
- entries are grouped by username
- new entries have: username, full_name, sum of retweets
- new entries are ordered in descending order

>SELECT username, full_name,  SUM(retweets) as retweets FROM tweet GROUP BY username ORDER BY retweets DESC;


## Users tweets
As for retweets, csv about tweets per user was made using SQL:
- entries are grouped by username
- new entries have: username, full_name, count of unique tweets
- new entries are ordered in descending order

>SELECT username, full_name,  COUNT(DISTINCT text) as tweets FROM tweet GROUP BY username ORDER BY tweets DESC;


## Average user's tweets length
As before, in these case SQL is a friend:
- entries are grouped by username
- new entries have: username, full_name, average words count
- new entries are ordered in descending order

>SELECT username, full_name, AVG(words) as words_avg FROM tweet GROUP BY username ORDER BY words_avg DESC;

## Grouped data to csv
To write a csv containing the grouped data; each query has been used to create a view and then export it to csv usind PyCharm.